## CIFAR-10 Training

This is an example of how to train the ViT on the CIFAR-10 dataset.

Since no kind of regularization, augmentation, etc. is applied, the model is highly overfitting (i.e., 100% accuracy on the training subset after 24 epochs). 


### Imports

In [1]:
import random

import torch
import torchvision
import torchvision.transforms as transforms

from tqdm import tqdm

from ViT import ViT

Set seeds

In [ ]:
seed = 42
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = True

### Load CIFAR dataset

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=64, shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(
    testset, batch_size=64, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


### Create instances that will be used during training.

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
epochs = 24
learning_rate = 1e-4
image_dims = (3, 32, 32)
num_patches = 8
embedding_dim = 256 # 512
mlp_hidden_dim = 256 # 128
num_heads = 16
output_dim = 10
num_blocks = 12

model = ViT(image_dims=image_dims, 
            num_heads=num_heads, 
            num_patches=num_patches, 
            embedding_dim=embedding_dim, 
            num_blocks=num_blocks, 
            output_dim=output_dim).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.1)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=8)
loss_fn = torch.nn.CrossEntropyLoss()

### Training loop

In [5]:
for epoch in range(epochs):
    train_loss = 0
    correct, total = 0, 0
    for batch in tqdm(train_loader, desc=f'Epoch: {epoch + 1} in training'):
        imgs, labels = batch
        imgs = imgs.to(device)
        labels = labels.to(device)
        preds = model(imgs)
        loss = loss_fn(preds, labels)

        train_loss += loss.detach().cpu().item() / len(train_loader)

        correct += torch.sum(torch.argmax(preds, dim=1)
                                 == labels).detach().cpu().item()
        total += len(imgs)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        optimizer.step()
    lr_scheduler.step()

    print(f"Epoch {epoch + 1} / {epochs} loss: {train_loss:.2f} accuracy: {correct / total * 100:.2f}%")

    with torch.no_grad():
        correct, total = 0, 0
        test_loss = 0
        for batch in tqdm(test_loader, desc='Testing'):
            imgs, labels = batch
            imgs = imgs.to(device)
            labels = labels.to(device)

            preds = model(imgs)
            loss = loss_fn(preds, labels)
            test_loss += loss.detach().cpu().item() / len(test_loader)
            correct += torch.sum(torch.argmax(preds, dim=1)
                                 == labels).detach().cpu().item()
            total += len(imgs)
        print(f"Test loss: {test_loss:.2f} accuracy: {correct / total * 100:.2f}%")

Epoch: 1 in training: 100%|██████████| 782/782 [00:33<00:00, 23.32it/s]


Epoch 1 / 24 loss: 1.61 accuracy: 41.20%


Testing: 100%|██████████| 157/157 [00:06<00:00, 26.08it/s]


Test loss: 1.44 accuracy: 49.33%


Epoch: 2 in training: 100%|██████████| 782/782 [00:33<00:00, 23.15it/s]


...

Test loss: 2.73 accuracy: 69.10%


Epoch: 24 in training: 100%|██████████| 782/782 [00:33<00:00, 23.17it/s]


Epoch 24 / 24 loss: 0.00 accuracy: 100.00%


Testing: 100%|██████████| 157/157 [00:06<00:00, 25.88it/s]

Test loss: 2.76 accuracy: 69.25%
